In [ ]:
import numpy as np
import argparse
import os
import imp
import re
import pickle
import datetime
import random
import math
import copy

from sklearn.model_selection import KFold, StratifiedKFold, StratifiedShuffleSplit
import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils import data
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset, Subset

from utils import metrics


In [ ]:
def predict_all_visits_mse_loss(y_pred, y_true, x_lab_length):
    batch_size = len(y_true)
    loss = torch.nn.MSELoss()
    indices = torch.arange(batch_size, dtype=torch.int64)
    losses = 0
    for i in indices:
        losses += loss(y_pred[i][:x_lab_length[i].long()], y_true[i][:x_lab_length[i].long()])
    return losses/batch_size

In [ ]:
class Dataset(data.Dataset):
    def __init__(self, x, y, x_lab_length):
        self.x= x
        self.y = y
        self.x_lab_length = x_lab_length
        
    def __getitem__(self, index): # 返回的是tensor
        return self.x[index], self.y[index], self.x_lab_length[index]

    def __len__(self):
        return len(self.y)

In [ ]:
data_path = '../dataset/tongji/processed_data/'
file_name = './ckpt/gru.pth'

batch_size = 64
num_epochs = 100
device = torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu')
# device = torch.device('cpu')

data_path = "../dataset/tongji/processed_data/"
x = pickle.load(open(data_path+"x.pkl", "rb"))
y = pickle.load(open(data_path+"y.pkl", "rb"))
x_lab_length = pickle.load(open(data_path+"visits_length.pkl", "rb"))
train_dataset = Dataset(x, y, x_lab_length)


In [ ]:
class Transformer(nn.Module):
    def __init__(self, input_lab_dim, input_demo_dim, max_visits, hidden_dim, output_dim, act_layer=nn.GELU, drop=0.):
        super(Transformer, self).__init__()

        # hyperparameters
        self.input_lab_dim = input_lab_dim
        self.input_demo_dim = input_demo_dim
        self.max_visits = max_visits
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.proj = nn.Linear(input_demo_dim+input_lab_dim, hidden_dim)
        self.bn = nn.BatchNorm1d(max_visits)
        
        self.gru = nn.GRU(input_size = hidden_dim, hidden_size = hidden_dim, num_layers = 1, batch_first = True)

        self.encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=4, dim_feedforward=512, activation='gelu')
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=1)
        
        self.act = act_layer()
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.drop = nn.Dropout(drop)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.proj(x)
        # x = self.act(x)
        # x = self.bn(x)

        # x, _ = self.gru(x)
        x = self.transformer_encoder(x)

        x = self.drop(x)
        x = self.fc(x)
        x = self.drop(x)

        # x = self.sigmoid(x)
        return x


In [ ]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED) # numpy
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED) # cpu
torch.cuda.manual_seed(RANDOM_SEED) # gpu
torch.backends.cudnn.deterministic=True # cudnn
np.set_printoptions(threshold=np.inf, precision=2, suppress=True)


def train_epoch(model, device, dataloader, loss_fn, optimizer):
    train_loss = []
    model.train()
    for step, data in enumerate(dataloader):   
        batch_x, batch_y, batch_x_lab_length = data
        batch_x, batch_y, batch_x_lab_length = batch_x.float(), batch_y.float(), batch_x_lab_length.float()
        batch_y = batch_y[:,:,1] # 0: outcome, 1: los
        batch_y = batch_y.unsqueeze(-1)
        optimizer.zero_grad()
        output = model(batch_x)
        loss = loss_fn(output, batch_y, batch_x_lab_length)
        train_loss.append(loss.item())
        loss.backward()
        optimizer.step()
    return np.array(train_loss).mean()

def val_epoch(model, device, dataloader, loss_fn):
    """
    val / test
    """
    val_loss = []
    y_pred = []
    y_true = []
    model.eval()
    with torch.no_grad():
        for step, data in enumerate(dataloader):   
            batch_x, batch_y, batch_x_lab_length = data
            batch_x, batch_y, batch_x_lab_length = batch_x.float(), batch_y.float(), batch_x_lab_length.float()
            batch_y = batch_y[:,:,1] # 0: outcome, 1: los
            batch_y = batch_y.unsqueeze(-1)
            output = model(batch_x)
            loss = loss_fn(output, batch_y, batch_x_lab_length)
            val_loss.append(loss.item())
            output = torch.squeeze(output)
            batch_y = torch.squeeze(batch_y)
            for i in range(len(batch_y)):
                y_pred.extend(output[i][:batch_x_lab_length[i].long()].tolist())
                y_true.extend(batch_y[i][:batch_x_lab_length[i].long()].tolist())
    evaluation_scores = metrics.print_metrics_regression(y_true, y_pred, verbose=1)
    return np.array(val_loss).mean(), evaluation_scores

In [ ]:
model = Transformer(input_lab_dim=25, input_demo_dim=2, max_visits=13, hidden_dim=32, output_dim=1, act_layer=nn.GELU, drop=0.).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = predict_all_visits_mse_loss
train_loader = DataLoader(train_dataset, batch_size=128)
train_loss = train_epoch(model, device, train_loader, criterion, optimizer)
val_loss = val_epoch(model, device, train_loader, criterion)

In [ ]:
all_history={}
test_performance={'test_loss': [],'test_mad':[], 'test_mse':[], 'test_mape':[]}
dataset = train_dataset
num_folds = 10
kfold_test = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=RANDOM_SEED)
skf = kfold_test.split(np.arange(len(dataset)), dataset.y[:, 0, 0])
for fold_test, (train_and_val_idx, test_idx) in enumerate(skf):
    print('====== Test Fold {} ======'.format(fold_test + 1))
    sss = StratifiedShuffleSplit(
        n_splits=1, test_size=1 / (num_folds - 1), random_state=RANDOM_SEED
    )

    test_sampler = SubsetRandomSampler(test_idx)
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)
    sub_dataset = Dataset(dataset.x[train_and_val_idx], dataset.y[train_and_val_idx], dataset.x_lab_length[train_and_val_idx])
    all_history['test_fold_{}'.format(fold_test+1)] = {}

    train_idx, val_idx = next(
        sss.split(np.arange(len(train_and_val_idx)), sub_dataset.y[:, 0, 0])
    )

    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)
    model = Transformer(input_lab_dim=25, input_demo_dim=2, max_visits=13, hidden_dim=32, output_dim=1, act_layer=nn.GELU, drop=0.).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = predict_all_visits_mse_loss
    history = {'train_loss': [], 'val_loss': [],'val_mad':[], 'val_mse':[], 'val_mape':[]}
    best_val_performance = 1e8
    for epoch in range(num_epochs):
        train_loss=train_epoch(model, device, train_loader, criterion, optimizer)
        val_loss, val_evaluation_scores=val_epoch(model, device, val_loader, criterion)
        # save performance history on validation set
        print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Val Loss:{:.3f}".format(epoch + 1, num_epochs, train_loss, val_loss))
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_mad'].append(val_evaluation_scores['mad'])
        history['val_mse'].append(val_evaluation_scores['mse'])
        history['val_mape'].append(val_evaluation_scores['mape'])  
        # if mad is lower, than set the best mad, save the model, and test it on the test set
        if val_evaluation_scores['mad'] < best_val_performance:
            best_val_performance = val_evaluation_scores['mad']
            torch.save(model.state_dict(), './checkpoints/transformer_los_best_mad.pth')
    all_history['test_fold_{}'.format(fold_test+1)] = history  
    print(
        f"Best performance on val set {fold_test+1}: \
        MAD = {best_val_performance}"
    )
    model = Transformer(input_lab_dim=25, input_demo_dim=2, max_visits=13, hidden_dim=32, output_dim=1, act_layer=nn.GELU, drop=0.).to(device)
    model.load_state_dict(torch.load('./checkpoints/transformer_los_best_mad.pth'))
    test_loss, test_evaluation_scores = val_epoch(model, device, test_loader, criterion)
    test_performance['test_loss'].append(test_loss)
    test_performance['test_mad'].append(test_evaluation_scores['mad'])
    test_performance['test_mse'].append(test_evaluation_scores['mse'])
    test_performance['test_mape'].append(test_evaluation_scores['mape'])
    print(f"Performance on test set {fold_test+1}: MAD = {test_evaluation_scores['mad']}, MSE = {test_evaluation_scores['mse']}, MAPE = {test_evaluation_scores['mape']}")


In [ ]:
# Calculate average performance on 10-fold test set
# print(test_performance)
test_mad_list = np.array(test_performance['test_mad'])
test_mse_list = np.array(test_performance['test_mse'])
test_mape_list = np.array(test_performance['test_mape'])
print('MAD: mean={:.3f}, std={:.3f}'.format(test_mad_list.mean(), test_mad_list.std()))
print('MSE: mean={:.3f}, std={:.3f}'.format(test_mse_list.mean(), test_mse_list.std()))
print('MAPE: mean={:.3f}, std={:.3f}'.format(test_mape_list.mean(), test_mape_list.std()))

result = """
- Transformer
MAD: mean=4.607, std=0.525
MSE: mean=41.819, std=9.241
MAPE: mean=417.102, std=138.482

- GRU
MAD: mean=4.127, std=0.452
MSE: mean=36.479, std=8.446
MAPE: mean=351.916, std=115.702
"""